# `gymnasium.Env`类

`gymnasium.Env`类是在Gymnasium库中用于实现强化学习智能体环境的主要类。这个类封装了具有任意后台动态的环境，通过`step()`和`reset()`函数进行操作。环境可以被单个智能体部分或完全观察。对于多智能体环境，请参考PettingZoo库。

### 主要API方法

以下是使用`gymnasium.Env`类时需要知道的主要API方法：

- **step(action)**: 这个方法用于更新环境状态，基于智能体执行的动作，并返回以下四个值：
  - 下一个智能体观察结果；
  - 执行该动作获得的奖励；
  - 环境是否已终止；
  - 环境是否由于最新动作被截断；
  - 关于这一步的环境信息，如度量指标、调试信息等。

- **reset()**: 重置环境到初始状态，开始新的回合之前必须调用。返回第一个智能体观察结果和其他信息，如度量指标、调试信息等。

- **render(mode='human')**: 渲染环境以帮助可视化智能体所看到的内容。常见的渲染模式包括“human”（人类观看）、“rgb_array”（返回RGB数组）、“ansi”（文本模式）。

- **close()**: 关闭环境，特别是当使用外部软件（如pygame进行渲染、数据库等）时非常重要。

### 附加属性

环境还具有一些额外的属性，帮助用户了解其实现：

- **action_space**: 对应于有效动作的`Space`对象，所有有效的动作都应该包含在这个空间内。

- **observation_space**: 对应于有效观察结果的`Space`对象，所有有效的观察结果都应该包含在这个空间内。

- **spec**: 环境规范，包含用于从`gymnasium.make()`初始化环境的信息。

- **metadata**: 环境的元数据，如渲染模式、渲染帧率等。

- **np_random**: 环境的随机数生成器。这在`super().reset(seed=seed)`调用和访问`np_random`时自动分配。

### 应用案例

假设我们要创建一个简单的迷宫游戏环境，智能体的目标是从起点到达终点。我们将实现`step()`、`reset()`、`render()`和`close()`方法，并定义`action_space`和`observation_space`。

```python
import gymnasium
from gymnasium import spaces
import numpy as np

class SimpleMazeEnv(gymnasium.Env):
    metadata = {'render.modes': ['human', 'ansi']}
    
    def __init__(self):
        super(SimpleMazeEnv, self).__init__()
        self.action_space = spaces.Discrete(4)  # 上、下、左、右
        self.observation_space = spaces.Box(low=0, high=4, shape=(2,), dtype=np.int)  # 位置(x, y)
        self.state = None

    def step(self, action):
        # 更新状态逻辑
        reward = -1  # 每步默认奖励
        terminated = False
        info = {}
        # 假设终点为(4, 4)，到达则结束
        if self.state == (4, 4):
            terminated = True
            reward = 10  # 到达终点奖励
        return self.state, reward, terminated, False, info

    def reset(self):
        self.state = (0, 0)  # 起始位置
        return self.state, {}

    def render(self, mode='human'):
        if mode == 'human':
            print(f"Agent Position: {self.state}")
        elif mode == 'ansi':
            return f"Agent Position: {self.state}"



    def close(self):
        pass
```

在这个案例中，我们定义了一个简单的迷宫环境，其中包含动作空间、观察空间和基本的环境逻辑。这个示例提供了创建自定义Gymnasium环境的基础框架，可以根据需要扩展和修改。

# `Methods` 方法

## （1） 训练步 `Env.step(action: ActType)` 

`Env.step(action: ActType)` 方法是强化学习中环境与智能体互动的核心。这个方法根据智能体提供的动作执行一个时间步（timestep）的环境动态，并返回一组值，代表这一动作的后果。在版本0.26之后，Gymnasium API 对这个方法进行了更新，用`terminated`和`truncated`替换了之前的`done`信号，以便更清晰地向用户传达环境结束的原因，这对于强化学习的自举算法至关重要。

### 参数

- **action (ActType)**: 智能体提供的动作，用于更新环境状态。动作的具体类型和结构取决于环境的`action_space`。

### 返回值

- **observation (ObsType)**: 作为智能体动作的结果的下一个观察值，它是环境`observation_space`的一个元素。例如，在CartPole游戏中，观察值可能是一个包含杆的位置和速度的NumPy数组。

- **reward (SupportsFloat)**: 由于采取该动作而获得的奖励。奖励是一个浮点数，表示智能体的行为有多好。

- **terminated (bool)**: 表示智能体是否达到终止状态（根据任务的马尔可夫决策过程（MDP）定义），可以是正面的或负面的。例如，达到目标状态或移动到Sutton和Barton的Gridworld中的熔岩中。如果为真，用户需要调用`reset()`。

- **truncated (bool)**: 表示是否满足MDP范围之外的截断条件。通常，这是一个时间限制，但也可以用来指示智能体物理地超出边界。它可以用来在达到终止状态之前提前结束回合。如果为真，用户需要调用`reset()`。

- **info (dict)**: 包含辅助诊断信息（有助于调试、学习和记录）。例如，可能包含描述智能体性能状态的指标、对观察结果隐藏的变量，或组合产生总奖励的各个奖励项。在OpenAI Gym v26之前，它包含`"TimeLimit.truncated"`来区分截断和终止，但在v26及以后版本中，这已被弃用，转而返回`terminated`和`truncated`变量。

### 应用案例

假设我们有一个智能体在CartPole环境中学习保持杆子平衡。以下是如何使用`step()`方法在环境中执行动作和接收反馈的示例：

```python
import gymnasium as gym

# 创建环境
env = gym.make('CartPole-v1')
# 初始化环境，获取初始观察值
observation = env.reset()

for _ in range(1000):  # 限制回合内的时间步数为1000
    env.render()  # 可视化环境状态
    action = env.action_space.sample()  # 随机选择一个动作
    observation, reward, terminated, truncated, info = env.step(action)  # 执行动作

    # 检查回合是否结束
    if terminated or truncated:
        break

env.close()  # 关闭环境
```

在这个案例中，智能体在每个时间步随机选择一个动作，并通过`step()`方法与环境互动，直到回合结束（由于达到终止状态或时间限制）。每次调用`step()`方法后，我们可以获取新的观察值、奖励以及回合是否结束的信息，从而指导智能体的学习过程。


### 应用案例2：使用简单的随机策略

#### FrozenLake-v1 环境

假设我们正在使用Gymnasium库的`FrozenLake-v1`环境，这是一个简单的网格世界游戏，智能体需要从起点走到终点，避免掉入洞里。我们将使用一个简单的策略来尝试解决这个问题，并展示`step()`方法返回的各个参数是如何在实际中使用的。在`FrozenLake-v1`中，`observation`是智能体在网格中的位置（以状态编号表示），`reward`是到达终点时给予的奖励，`terminated`表示游戏是否结束（成功到达终点或掉入洞中），`truncated`表示游戏是否因为达到步数限制而提前结束，`info`可能包含一些额外的环境信息，但在许多环境中，它可能不包含对决策有用的信息。

```python
import gymnasium as gym
import numpy as np

# 创建环境
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)

# 初始化环境和一些变量
state = env.reset()
terminated = False
total_reward = 0

while not terminated:
    env.render()  # 显示环境的当前状态

    # 随机选择一个动作
    action = env.action_space.sample()

    # 执行动作
    next_state, reward, terminated, truncated, info = env.step(action)
    
    # 更新总奖励
    total_reward += reward
    
    print(f"Action: {action}, State: {state} -> {next_state}, Reward: {reward}, Terminated: {terminated}")
    
    # 更新状态
    state = next_state

print(f"Episode ended with total reward: {total_reward}")

env.close()
```

在这个例子中，智能体在每个时间步随机选择一个动作（上、下、左、右），然后执行这个动作。每次调用`step(action)`时，环境根据智能体的动作更新其状态，并返回新的状态（`next_state`）、智能体因该动作而获得的即时奖励（`reward`）、游戏是否结束（`terminated`）、游戏是否因达到步数限制而提前结束（`truncated`），以及其他可能的诊断信息（`info`）。

### 注意事项

- `reward`是智能体在执行动作后获得的即时反馈。在`FrozenLake-v1`中，只有在成功到达终点时才会获得奖励。
- `terminated`指示游戏是否已经结束，这可能是因为智能体成功到达了终点或者掉入了洞中。
- `truncated`在这个环境中通常为`False`，因为`FrozenLake-v1`通常不会设置步数限制。

通过这个案例，我们可以看到如何在实践中使用`Env.step()`方法以及如何处理它返回的信息来指导智能体的学习和决策过程。

## （2）重置 

Env.reset(*, seed: int | None = None, options: dict[str, Any] | None = None) → tuple[ObsType, dict[str, Any]]

`Env.reset()` 方法是用于将环境重置到初始状态的关键函数。在开始新的回合（episode）之前，或者在环境终止后想要开始新回合时，需要调用此方法。它不仅重置环境的内部状态，还返回初始观察值和可能的额外信息（`info`）。从版本v0.25开始，`reset()`方法的`return_info`参数被移除，现在总是期望返回`info`。

### 参数

- **seed (optional int)**: 用于初始化环境的伪随机数生成器（PRNG，即`np_random`）的种子。如果环境已经有了PRNG且调用`reset()`时`seed=None`（默认选项），则PRNG不会被重置。如果传入一个整数，即使环境已经有了PRNG，也会被重置。通常，在环境初始化后立即传入一个整数种子，然后不再改变。

- **options (optional dict)**: 指定如何重置环境的附加信息（可选，具体取决于环境）。这可以用于传递特定于环境的重置选项，例如初始化状态的配置。

### 返回值

- **observation (ObsType)**: 初始状态的观察值，它是`observation_space`的一个元素（通常是一个NumPy数组），类似于`step()`方法返回的观察值。

- **info (dictionary)**: 该字典包含补充观察值的辅助信息，应类似于`step()`方法返回的`info`。这可能包括有关环境初始化状态的额外诊断信息或其他相关指标。

### 应用案例

假设我们正在使用`FrozenLake-v1`环境，并希望在每次开始新回合时重置环境，并记录初始观察值和任何附加信息。

```python
import gymnasium as gym

# 创建环境
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)

# 重置环境并指定种子以保证可重复性
observation, info = env.reset(seed=42)

# 打印初始观察值和信息
print(f"Initial observation: {observation}")
print(f"Initial info: {info}")

env.close()
```

在这个示例中，我们通过传递`seed`参数来重置环境，确保每次环境的初始状态是可重复的，从而有助于调试或系统评估。`observation`提供了初始状态的观察值，`info`可能包含一些补充的初始状态信息，虽然在许多环境中`info`可能为空或仅包含少量信息。

### 注意事项

- 在自定义环境中，重写`reset()`方法时，应该在方法的第一行调用`super().reset(seed=seed)`以正确实现种子设定。
- 在多次实验或比较不同策略时，通过在`reset()`中使用固定的种子可以保证环境的初始状态一致，从而确保实验的公平性和可重复性。
- `options`参数允许为环境重置提供更多的灵活性，但其使用取决于特定环境的实现细节。

## （3）渲染

`Env.render()` 方法用于计算和显示环境的渲染帧，这些帧的渲染方式是在环境初始化时通过`render_mode`参数指定的。这个方法使得我们能够以不同的方式观察和展示环境状态，例如，通过图像（在图形界面中）或文本（在终端中）的形式。

### 渲染模式

环境的元数据`render_modes`（`env.metadata["render_modes"]`）包含了实现渲染模式的可能方式。大多数渲染模式的列表版本可以通过`gymnasium.make`自动应用包装器来收集渲染帧实现。

按照约定，如果`render_mode`是：

- **None**（默认）：不计算渲染。
- **"human"**：环境在当前显示器或终端中连续渲染，通常用于人类观看。这种渲染应该在`step()`过程中发生，因此不需要调用`render()`。返回`None`。
- **"rgb_array"**：返回单个帧，代表环境的当前状态。帧是一个形状为`(x, y, 3)`的`np.ndarray`，代表一个x乘y像素图像的RGB值。
- **"ansi"**：返回一个字符串（`str`）或`StringIO.StringIO`对象，包含每个时间步的终端风格文本表示。文本可以包含换行符和ANSI转义序列（例如，用于颜色）。
- **"rgb_array_list"** 和 **"ansi_list"**：通过在`gymnasium.make`过程中自动应用的包装器`gymnasium.wrappers.RenderCollection`，可以实现渲染模式的列表版本（除了"human"）。调用`render()`或`reset()`后，收集到的帧会被弹出。

### 注意事项

- 在自定义环境中实现`render()`方法时，确保类的元数据`"render_modes"`键包含支持的模式列表。
- 从版本0.25.0开始，`render`函数不再接受参数，而是应该在环境初始化时指定这些参数，例如，`gymnasium.make("CartPole-v1", render_mode="human")`。

### 应用案例

假设我们正在使用`CartPole-v1`环境，并希望在训练过程中以图像的形式观察智能体的表现。

```python
import gymnasium as gym

# 初始化环境并指定渲染模式为"rgb_array"
env = gym.make("CartPole-v1", render_mode="rgb_array")

# 重置环境，开始新的回合
observation, info = env.reset()

done = False
while not done:
    # 渲染环境状态，并返回当前帧
    frame = env.render()

    # 选择动作（这里使用随机动作作为示例）
    action = env.action_space.sample()

    # 执行动作，更新环境状态
    observation, reward, terminated, truncated, info = env.step(action)

env.close()  # 关闭环境
```

在这个示例中，我们通过设置`render_mode="rgb_array"`，每次调用`env.render()`时都能获取环境当前状态的图像帧。这使得我们能够在训练或评估过程中以视觉方式观察智能体的行为。

## （4）关闭

`Env.close()` 方法是在用户完成环境使用后进行“清理”工作的重要部分。这个方法对于关闭渲染窗口、数据库或HTTP连接至关重要。如果环境已经关闭，再次调用`close()`方法不会有任何效果，也不会引发错误。

### 关键点

- **资源释放**：在许多环境中，尤其是那些涉及图形渲染（如使用Pygame或OpenGL）、开放数据库连接或维持网络连接的环境，`close()`方法负责释放这些资源，确保程序优雅地结束，不会遗留开放的资源或连接。

- **重复调用安全**：`Env.close()`设计为幂等的，意味着多次调用不会产生副作用，这样可以确保用户在环境清理方面的代码更加健壮。

- **在脚本结束时调用**：为了避免资源泄露，建议在每次环境使用完毕后调用`close()`，尤其是在脚本或程序结束时。

### 应用案例

假设你正在使用`CartPole-v1`环境进行一些实验，并且在实验中使用了环境的渲染功能。以下是如何正确关闭环境的示例：

```python
import gymnasium as gym

# 初始化环境
env = gym.make("CartPole-v1", render_mode="human")

# 重置环境
env.reset()

# 假设进行一些实验，比如运行几个回合
for _ in range(10):
    action = env.action_space.sample()  # 随机选择动作
    env.step(action)  # 执行动作
    env.render()  # 渲染当前状态

# 实验完成后，关闭环境
env.close()
```

在这个示例中，`env.render()`方法用于渲染环境的当前状态，允许用户以图形方式观察智能体的行为。一旦完成了实验，调用`env.close()`以确保所有与环境相关的资源（如渲染窗口）被正确释放，防止资源泄露。

### 注意事项

- 即使你的环境可能不涉及显式的资源分配（如渲染窗口），仍然推荐在结束使用后调用`close()`方法，以保持代码的一致性和良好实践。
- 在使用包含多个环境的复杂设置（例如，同时运行多个环境实例）时，确保为每个环境实例调用`close()`方法，避免资源泄露。

# Env属性

## Env.action_space 和Env.observation_space

`Env.action_space` 属性定义了智能体可以在环境中执行的所有有效动作的空间。这个空间对象指明了动作的类型（如离散、连续等）和范围。通过了解`action_space`，智能体可以知道它可以采取哪些动作以及如何构造这些动作。同样的，`Env.observation_space` 定义了智能体可以观察到的环境状态的空间和范围。

### action_space

- **类型和范围**：`action_space`是一个`Space`对象，可能的类型包括`Discrete`（离散空间）、`Box`（连续空间）、`MultiDiscrete`、`MultiBinary`等。
- **离散动作空间**（`Discrete`）：如果`action_space`是`Discrete(n)`类型，这意味着有`n`个离散的动作可用，从0到`n-1`。例如，`Discrete(2)`表示有两个有效的离散动作：0和1。
- **连续动作空间**（`Box`）：如果`action_space`是`Box`类型，这表示动作是连续的，每个动作的取值范围由`Box`的上下界定义。例如，一个机器人臂的关节角度可能是一个`Box`空间。

### observation_space

- **状态的表示**：`observation_space`定义了智能体可以观察到的环境状态的范围和类型。这个空间对象帮助智能体了解它可以接收到哪些类型的观察值，以及这些观察值的维度和范围。
- **连续状态空间**（`Box`）：例如，`Box(-inf, inf, (4,), float32)`表示观察值是一个4维的浮点数向量，每个维度的取值范围是无限的。

### 应用案例

假设你正在开发一个智能体来学习玩CartPole游戏，你需要根据`action_space`和`observation_space`来设计智能体的动作选择和状态处理策略。

```python
import gymnasium as gym

# 初始化环境
env = gym.make('CartPole-v1')

# 检查动作空间和观察空间
print("Action Space:", env.action_space)  # 例如：Discrete(2)
print("Observation Space:", env.observation_space)  # 例如：Box(-inf, inf, (4,), float32)

# 使用动作空间
action = env.action_space.sample()  # 随机选择一个动作
observation, reward, terminated, truncated, info = env.step(action)  # 执行动作

env.close()
```

在这个例子中，我们首先检查了`CartPole-v1`环境的`action_space`和`observation_space`来了解智能体可用的动作类型和观察值的结构。然后，我们使用`env.action_space.sample()`方法随机选择一个有效动作来执行，并通过`env.step(action)`方法来更新环境状态并获取下一步的观察值。

通过理解`action_space`和`observation_space`，你可以更好地设计智能体的决策逻辑，使其能够在给定环境中有效地学习和操作。

## Env.metadata

`Env.metadata` 属性是一个字典，包含了环境的元数据信息，如渲染模式、渲染帧率（fps）等。这些信息对于了解环境的支持功能和配置渲染参数非常有用。

### 主要内容

- **render_modes**: 列出了环境支持的渲染模式，例如`['human', 'rgb_array', 'ansi']`。这些模式决定了`render()`方法的输出类型，例如直接在屏幕上渲染环境状态、返回环境状态的RGB数组，或以文本形式返回环境状态。
- **render_fps**: 指定了渲染时的帧率，这对于控制渲染速度和平滑度很重要，尤其是在`'human'`模式下。

### 应用案例

假设我们想要了解一个新环境支持哪些渲染模式，并根据这些信息来选择合适的渲染方式。

```python
import gymnasium as gym

# 创建环境实例
env = gym.make('CartPole-v1')

# 访问环境的元数据
metadata = env.metadata
print("Supported render modes:", metadata['render_modes'])

# 根据支持的渲染模式选择一种进行渲染
if 'rgb_array' in metadata['render_modes']:
    env.reset()
    frame = env.render(mode='rgb_array')  # 获取环境状态的RGB数组
    # 可以进一步处理frame，例如显示或保存
elif 'human' in metadata['render_modes']:
    env.reset()
    env.render(mode='human')  # 直接在屏幕上渲染环境状态
else:
    print("No supported render mode found for visualization.")

env.close()
```

在这个示例中，我们首先检查了`CartPole-v1`环境的`metadata`属性来确定它支持哪些渲染模式。然后，根据这些信息，我们选择了一种合适的渲染模式来可视化环境状态。如果环境支持`'rgb_array'`模式，我们就获取环境状态的RGB数组；如果支持`'human'`模式，我们就直接在屏幕上渲染环境状态。

### 注意事项

- 在设计和测试智能体时，了解和利用环境的`metadata`可以帮助你更好地理解和控制环境的行为，特别是在环境渲染和可视化方面。
- 某些环境可能不支持所有渲染模式，因此在尝试渲染之前检查`metadata['render_modes']`是一个好习惯。

## Env.render_mode

`Env.render_mode` 属性指定了环境初始化时确定的渲染模式。这个模式决定了环境状态如何被可视化和渲染，比如是否在屏幕上直接显示环境状态（`'human'`模式），或者是以编程方式获取环境状态的图像表示（`'rgb_array'`模式），或者其他支持的渲染模式。

### 渲染模式选项

- **None**（默认）：如果`render_mode`设置为`None`，则不会计算任何渲染输出。这种模式适用于只关心环境动态而不需要可视化的场景。
- **'human'**：在这种模式下，环境状态会被渲染到屏幕上，以便人类观察。这通常意味着使用图形窗口来显示环境。
- **'rgb_array'**：环境状态将被渲染为RGB图像数组，可以用于进一步处理或存储。这对于自动处理和分析渲染输出非常有用。
- **'ansi'**：环境状态以ANSI字符串的形式返回，通常用于文本环境或需要在终端中显示状态的场景。

### 设置渲染模式

渲染模式在环境初始化时通过`gymnasium.make()`函数的`render_mode`参数设置。例如：

```python
import gymnasium as gym

# 创建环境并指定渲染模式为'rgb_array'
env = gym.make('CartPole-v1', render_mode='rgb_array')

# 重置环境，开始新的回合
observation = env.reset()

# 执行一些动作，并获取渲染的RGB图像
action = env.action_space.sample()
observation, reward, terminated, truncated, info = env.step(action)
frame = env.render()  # 在'rgb_array'模式下，env.render() 返回当前状态的RGB图像

env.close()  # 关闭环境
```

### 注意事项

- `render_mode`的设置在整个环境实例的生命周期中是固定的，不能在环境创建后更改。
- 在进行大规模训练或自动化测试时，建议将`render_mode`设置为`None`以提高性能，因为渲染操作可能会显著增加计算成本。
- 当使用`'human'`模式时，通常不需要在每个时间步调用`env.render()`，因为环境会自动更新屏幕上的显示。然而，对于其他模式（如`'rgb_array'`），可能需要在每个时间步调用`env.render()`来获取最新的渲染帧。

通过合理使用`Env.render_mode`属性，可以根据需要调整环境的渲染行为，无论是进行交互式观察还是自动化处理和分析。

## Env.spec
`Env.spec` 属性包含了环境的`EnvSpec`对象，这通常在通过`gymnasium.make()`函数创建环境时设置。`EnvSpec`对象提供了关于环境的元数据，比如环境的ID、奖励阈值、最大步数等，这对于理解环境的结构和期望的行为非常有用。

### 主要内容

`EnvSpec`对象包含的信息可能包括但不限于：

- **id**: 环境的唯一标识符，如`'CartPole-v1'`。
- **entry_point**: 创建环境实例的函数路径。
- **reward_threshold**: 达到该奖励阈值可能被认为是解决了环境的问题。
- **max_episode_steps**: 每个回合的最大步数限制。
- **nondeterministic**: 环境是否具有非确定性特征。
- **order_enforce**: 是否强制执行环境方法调用的顺序（如先reset后step）。
- **kwargs**: 创建环境时传递给`entry_point`的参数。

### 应用案例

假设我们想要加载一个特定的环境，并查看其规格详细信息，以便了解环境的特性和限制。

```python
import gymnasium as gym

# 创建环境实例
env = gym.make("CartPole-v1")

# 访问环境的规格信息
spec = env.spec

# 打印环境的规格详细信息
print(f"ID: {spec.id}")
print(f"Reward Threshold: {spec.reward_threshold}")
print(f"Max Episode Steps: {spec.max_episode_steps}")
print(f"Nondeterministic: {spec.nondeterministic}")
print(f"Order Enforce: {spec.order_enforce}")
print(f"Kwargs: {spec.kwargs}")

env.close()
```

在这个示例中，我们首先使用`gym.make()`函数加载了`CartPole-v1`环境，并通过访问`env.spec`来获取环境的规格信息。然后，我们打印了一些关键的规格信息，比如环境的ID、奖励阈值、每个回合的最大步数限制等。这些信息对于理解环境的行为和设计相应的智能体策略非常重要。

### 注意事项

- `Env.spec`在环境初始化过程中自动设置，通常不需要手动修改。
- 了解环境的规格可以帮助开发者更好地理解环境的约束和目标，以及如何调整智能体的训练策略来满足这些约束。
- 不是所有的环境都会设置所有`EnvSpec`的属性，部分信息可能根据环境的不同而有所差异。

## Env.unwrapped
`Env.unwrapped` 属性用于获取被多层包装器（wrappers）包裹的环境的基本（原始）版本。在Gymnasium中，环境可以通过添加包装器来扩展其功能，比如修改观察空间或动作空间、记录视频等。有时候，为了访问原始环境的特定方法或属性，需要去除这些包装器，这时`unwrapped`属性就非常有用。

### 使用方法

当你有一个被多个包装器包裹的环境时，可以通过调用`unwrapped`属性来获取最底层的环境实例。这允许我们直接访问和操作原始环境，而不受中间包装器的影响。

### 示例

假设我们创建了一个环境，并对其应用了多个包装器：

```python
import gymnasium as gym
from gymnasium.wrappers import TimeLimit, Monitor

# 创建环境并应用多个包装器
env = gym.make("CartPole-v1")
env = TimeLimit(env, max_episode_steps=100)
env = Monitor(env, "./video", video_callable=lambda episode_id: True, force=True)

# 使用包装器后的环境
print("Wrapped Environment:", type(env))

# 获取并使用原始环境
base_env = env.unwrapped
print("Base Environment:", type(base_env))
```

在这个示例中，我们首先创建了一个`CartPole-v1`环境，并通过应用`TimeLimit`和`Monitor`包装器来扩展其功能。然后，我们通过`env.unwrapped`获取了原始的`CartPole-v1`环境实例，并打印了其类型来验证我们已经成功地访问到了基本环境。

### 注意事项

- 使用`unwrapped`属性时要小心，因为这会绕过所有包装器提供的功能和安全检查。在一些情况下，直接操作基本环境可能会导致不符合预期的行为。
- `unwrapped`属性仅适用于那些实际上被包装器包裹的环境。如果环境没有被任何包装器包裹，`env.unwrapped`将简单地返回环境本身。
- 在开发环境时，确保原始环境提供了所需的所有方法和属性，特别是当我们打算使用`unwrapped`属性来绕过包装器时。

## Env.np_random

`Env.np_random` 属性提供了对环境内部使用的NumPy随机数生成器（`np.random.Generator`实例）的访问。这个随机数生成器用于在环境内部生成随机数，例如决定状态转移、奖励分配或任何其他随机行为。如果`_np_random`属性没有被显式设置，那么在第一次访问`np_random`时，它将使用一个随机种子进行初始化。

### 使用方法

通过访问`Env.np_random`，智能体或环境的开发者可以保证在环境内部产生的随机数是可重现的，只要在环境初始化或重置时提供相同的种子。这对于实验的重现性和调试是非常重要的。

### 示例

假设我们需要在自定义环境中使用`np_random`来随机选择初始状态或执行其他随机决策：

```python
import gymnasium as gym
import numpy as np

class CustomEnv(gym.Env):
    def __init__(self):
        super(CustomEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(2)  # 例如，有两个动作
        self.observation_space = gym.spaces.Discrete(5)  # 例如，有五个状态
        
    def reset(self, *, seed=None, return_info=False, options=None):
        super().reset(seed=seed)  # 正确初始化_np_random
        # 使用np_random选择一个初始状态
        initial_state = self.np_random.integers(low=0, high=self.observation_space.n)
        return initial_state, {}
    
    def step(self, action):
        # 示例：根据动作和当前状态随机决定下一个状态
        next_state = self.np_random.integers(low=0, high=self.observation_space.n)
        reward = self.np_random.random()  # 示例：随机生成一个奖励
        terminated = False  # 在这个示例中，我们不考虑终止条件
        truncated = False
        info = {}
        return next_state, reward, terminated, truncated, info

# 创建并测试自定义环境
env = CustomEnv()
env.reset(seed=42)  # 提供种子以确保随机性的可重现性
for _ in range(3):
    print(env.step(env.action_space.sample()))  # 执行并打印几步

env.close()
```

在这个示例中，我们创建了一个简单的自定义环境，其中包含了如何使用`Env.np_random`来生成随机初始状态、随机选择下一个状态和随机奖励。通过在`reset`方法中调用`super().reset(seed=seed)`，我们确保了环境的随机数生成器可以通过提供种子来正确初始化，从而使得环境的随机行为是可重现的。

### 注意事项

- 在设计和实现自定义环境时，通过使用环境的内部随机数生成器`np_random`而不是直接使用`np.random`，可以更好地控制环境的随机性，并保证实验的可重现性。
- 当需要重置环境的随机状态时（例如，在每次实验开始时），应该通过`reset`方法传递种子参数来实现，这样可以保证随机数生成器的状态也会相应重置。